In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Loading in data
cr = pd.read_csv('FIP_concat_CRE_to_JHU.csv')
# jhu = pd.read_csv('JHU_Covid_Cases_Processing_Complete.csv')
jhu_cd = pd.read_csv("covid_us_county.csv")
pd.set_option('display.max_rows', None)
# Realized that there may be some int and NANs that are in the CRE data with FIPS BUT NOT in the data I cleaened for JHU_Covid cases
# file. Went ahead and recleaend what was necessary to do a merge, then will drop NANs and any Unassigned values as I see them

In [3]:
cr.dtypes

Unnamed: 0                  int64
FIPS                        int64
State                      object
County                     object
POPUNI                      int64
Total_Population            int64
Zero_RF                     int64
One-Two_RF                  int64
Three_RF                    int64
Housing_Units               int64
Hispanic_Pop              float64
White_Pop                 float64
Black_Pop                 float64
Native_Pop                float64
Asian_Pop                 float64
Pacific_Islander_Pop      float64
Other_race_Pop            float64
Bi_Tri_Racial_Pop         float64
Male_Pop                  float64
Female_Pop                float64
Veteran                   float64
Gini_Ind_Income           float64
Rural_Pop                 float64
Median_Age_Pop            float64
Age_65+_Pop               float64
Disability_Pop            float64
Below_Poverty_Level       float64
Single_Mothers_Pop        float64
Single_Fathers_Pop        float64
1.51_plus_fami

In [4]:
jhu_cd.dtypes

fips          float64
county         object
state          object
lat           float64
long          float64
1/1/23          int64
1/2/23          int64
1/3/23          int64
1/4/23          int64
1/5/23          int64
1/6/23          int64
1/7/23          int64
1/8/23          int64
1/9/23          int64
1/10/23         int64
1/11/23         int64
1/12/23         int64
1/13/23         int64
1/14/23         int64
1/15/23         int64
1/16/23         int64
1/17/23         int64
1/18/23         int64
1/19/23         int64
1/23/23         int64
1/24/23         int64
1/25/23         int64
1/26/23         int64
1/27/23         int64
1/28/23         int64
1/29/23         int64
1/30/23         int64
1/31/23         int64
2/1/23          int64
2/2/23          int64
2/3/23          int64
2/4/23          int64
2/5/23          int64
2/6/23          int64
2/7/23          int64
2/8/23          int64
2/9/23          int64
2/10/23         int64
2/11/23         int64
2/12/23         int64
2/13/23   

In [5]:
jhu_cd = jhu_cd.groupby('fips').agg(
    {'cases': 'max', 'deaths': 'max', 'lat': 'max', 'long': 'max', 'county': 'max'})
jhu_cd.head()

,cases,deaths,lat,long,county
fips,,,,,
60.0,8320,34,-14.271000,-170.132000,NaN
66.0,60960,420,13.444300,144.793700,NaN
69.0,13605,41,15.097900,145.673900,NaN
78.0,24741,130,18.335800,-64.896300,NaN
1001.0,19732,230,32.539527,-86.644082,Autauga


In [6]:
# resetting index
jhu_cd.reset_index(inplace=True)

In [7]:
# renaming cols
jhu_cd = jhu_cd.rename(columns={'fips': 'FIPS', 'cases': 'Cases',
                       'deaths': 'Deaths', 'lat': 'Lat', 'long': 'Long', 'county': 'County'})
jhu_cd.head()

,FIPS,Cases,Deaths,Lat,Long,County
0,60.0,8320,34,-14.271000,-170.132000,NaN
1,66.0,60960,420,13.444300,144.793700,NaN
2,69.0,13605,41,15.097900,145.673900,NaN
3,78.0,24741,130,18.335800,-64.896300,NaN
4,1001.0,19732,230,32.539527,-86.644082,Autauga


In [8]:
mergedRes = pd.merge(jhu_cd, cr, on='FIPS')

In [9]:
mergedRes.head()

,FIPS,Cases,Deaths,Lat,Long,County_x,Unnamed: 0,State,County_y,POPUNI,...,Single_Fathers_Pop,1.51_plus_family_homes,HighSchool_Grad,Multilingual_5yrs_Plus,Full_time_workers,No_Health_Insur,Internet_homes,No_Vehicle,Homeowner_Vacancy,Rental_Vacancy
0,1001.0,19732,230,32.539527,-86.644082,Autauga,0,Alabama,Autauga County,55688,...,723.0,111.0,49283.0,779.0,29291.0,3953.0,44884.0,2450.0,779.0,1726.0
1,1003.0,69641,724,30.727750,-87.722071,Baldwin,1,Alabama,Baldwin County,221898,...,2218.0,887.0,201483.0,3994.0,112058.0,19748.0,181512.0,7322.0,7322.0,96747.0
2,1005.0,7451,103,31.868263,-85.387129,Barbour,2,Alabama,Barbour County,22023,...,220.0,132.0,16120.0,572.0,8170.0,2488.0,13323.0,2246.0,836.0,1629.0
3,1007.0,8067,109,32.996421,-87.125115,Bibb,3,Alabama,Bibb County,20393,...,346.0,163.0,16130.0,265.0,8096.0,2182.0,14111.0,1264.0,305.0,1142.0
4,1009.0,18616,261,33.982109,-86.567906,Blount,4,Alabama,Blount County,57697,...,1038.0,115.0,46446.0,1961.0,27463.0,6231.0,42118.0,2365.0,403.0,4846.0


In [10]:
# creating a drop column list and dropping
dcols = ['Unnamed: 0', 'County_x']
mergedRes.drop(columns=dcols, axis=1, inplace=True)

In [11]:
mergedRes.head()

,FIPS,Cases,Deaths,Lat,Long,State,County_y,POPUNI,Total_Population,Zero_RF,...,Single_Fathers_Pop,1.51_plus_family_homes,HighSchool_Grad,Multilingual_5yrs_Plus,Full_time_workers,No_Health_Insur,Internet_homes,No_Vehicle,Homeowner_Vacancy,Rental_Vacancy
0,1001.0,19732,230,32.539527,-86.644082,Alabama,Autauga County,55688,55380,20573,...,723.0,111.0,49283.0,779.0,29291.0,3953.0,44884.0,2450.0,779.0,1726.0
1,1003.0,69641,724,30.727750,-87.722071,Alabama,Baldwin County,221898,212830,78622,...,2218.0,887.0,201483.0,3994.0,112058.0,19748.0,181512.0,7322.0,7322.0,96747.0
2,1005.0,7451,103,31.868263,-85.387129,Alabama,Barbour County,22023,25361,5024,...,220.0,132.0,16120.0,572.0,8170.0,2488.0,13323.0,2246.0,836.0,1629.0
3,1007.0,8067,109,32.996421,-87.125115,Alabama,Bibb County,20393,22493,6280,...,346.0,163.0,16130.0,265.0,8096.0,2182.0,14111.0,1264.0,305.0,1142.0
4,1009.0,18616,261,33.982109,-86.567906,Alabama,Blount County,57697,57681,18189,...,1038.0,115.0,46446.0,1961.0,27463.0,6231.0,42118.0,2365.0,403.0,4846.0


In [12]:
JHU_CRE = mergedRes.rename(columns={'County_y': 'County'})

In [13]:
JHU_CRE.head()

,FIPS,Cases,Deaths,Lat,Long,State,County,POPUNI,Total_Population,Zero_RF,...,Single_Fathers_Pop,1.51_plus_family_homes,HighSchool_Grad,Multilingual_5yrs_Plus,Full_time_workers,No_Health_Insur,Internet_homes,No_Vehicle,Homeowner_Vacancy,Rental_Vacancy
0,1001.0,19732,230,32.539527,-86.644082,Alabama,Autauga County,55688,55380,20573,...,723.0,111.0,49283.0,779.0,29291.0,3953.0,44884.0,2450.0,779.0,1726.0
1,1003.0,69641,724,30.727750,-87.722071,Alabama,Baldwin County,221898,212830,78622,...,2218.0,887.0,201483.0,3994.0,112058.0,19748.0,181512.0,7322.0,7322.0,96747.0
2,1005.0,7451,103,31.868263,-85.387129,Alabama,Barbour County,22023,25361,5024,...,220.0,132.0,16120.0,572.0,8170.0,2488.0,13323.0,2246.0,836.0,1629.0
3,1007.0,8067,109,32.996421,-87.125115,Alabama,Bibb County,20393,22493,6280,...,346.0,163.0,16130.0,265.0,8096.0,2182.0,14111.0,1264.0,305.0,1142.0
4,1009.0,18616,261,33.982109,-86.567906,Alabama,Blount County,57697,57681,18189,...,1038.0,115.0,46446.0,1961.0,27463.0,6231.0,42118.0,2365.0,403.0,4846.0


In [14]:
JHU_CRE.isnull().sum()

FIPS                      0
Cases                     0
Deaths                    0
Lat                       0
Long                      0
State                     0
County                    0
POPUNI                    0
Total_Population          0
Zero_RF                   0
One-Two_RF                0
Three_RF                  0
Housing_Units             0
Hispanic_Pop              0
White_Pop                 0
Black_Pop                 0
Native_Pop                0
Asian_Pop                 0
Pacific_Islander_Pop      0
Other_race_Pop            0
Bi_Tri_Racial_Pop         0
Male_Pop                  0
Female_Pop                0
Veteran                   0
Gini_Ind_Income           0
Rural_Pop                 0
Median_Age_Pop            0
Age_65+_Pop               0
Disability_Pop            0
Below_Poverty_Level       0
Single_Mothers_Pop        0
Single_Fathers_Pop        0
1.51_plus_family_homes    0
HighSchool_Grad           0
Multilingual_5yrs_Plus    0
Full_time_workers   

In [15]:
JHU_CRE.isna().sum()

FIPS                      0
Cases                     0
Deaths                    0
Lat                       0
Long                      0
State                     0
County                    0
POPUNI                    0
Total_Population          0
Zero_RF                   0
One-Two_RF                0
Three_RF                  0
Housing_Units             0
Hispanic_Pop              0
White_Pop                 0
Black_Pop                 0
Native_Pop                0
Asian_Pop                 0
Pacific_Islander_Pop      0
Other_race_Pop            0
Bi_Tri_Racial_Pop         0
Male_Pop                  0
Female_Pop                0
Veteran                   0
Gini_Ind_Income           0
Rural_Pop                 0
Median_Age_Pop            0
Age_65+_Pop               0
Disability_Pop            0
Below_Poverty_Level       0
Single_Mothers_Pop        0
Single_Fathers_Pop        0
1.51_plus_family_homes    0
HighSchool_Grad           0
Multilingual_5yrs_Plus    0
Full_time_workers   

In [16]:
# Making sure none of the FIPS that contained "Out of" or "Unassigned" are in the county column
JHU_CRE = JHU_CRE[~JHU_CRE.County.str.startswith('Out of')]

In [17]:
# This is a good sign
print(len(JHU_CRE))

3142


In [18]:
JHU_CRE = JHU_CRE[JHU_CRE.County != 'Unassigned']

In [19]:
JHU_CRE.head()

,FIPS,Cases,Deaths,Lat,Long,State,County,POPUNI,Total_Population,Zero_RF,...,Single_Fathers_Pop,1.51_plus_family_homes,HighSchool_Grad,Multilingual_5yrs_Plus,Full_time_workers,No_Health_Insur,Internet_homes,No_Vehicle,Homeowner_Vacancy,Rental_Vacancy
0,1001.0,19732,230,32.539527,-86.644082,Alabama,Autauga County,55688,55380,20573,...,723.0,111.0,49283.0,779.0,29291.0,3953.0,44884.0,2450.0,779.0,1726.0
1,1003.0,69641,724,30.727750,-87.722071,Alabama,Baldwin County,221898,212830,78622,...,2218.0,887.0,201483.0,3994.0,112058.0,19748.0,181512.0,7322.0,7322.0,96747.0
2,1005.0,7451,103,31.868263,-85.387129,Alabama,Barbour County,22023,25361,5024,...,220.0,132.0,16120.0,572.0,8170.0,2488.0,13323.0,2246.0,836.0,1629.0
3,1007.0,8067,109,32.996421,-87.125115,Alabama,Bibb County,20393,22493,6280,...,346.0,163.0,16130.0,265.0,8096.0,2182.0,14111.0,1264.0,305.0,1142.0
4,1009.0,18616,261,33.982109,-86.567906,Alabama,Blount County,57697,57681,18189,...,1038.0,115.0,46446.0,1961.0,27463.0,6231.0,42118.0,2365.0,403.0,4846.0


In [20]:
# checking columns
JHU_CRE.columns

Index(['FIPS', 'Cases', 'Deaths', 'Lat', 'Long', 'State', 'County', 'POPUNI',
       'Total_Population', 'Zero_RF', 'One-Two_RF', 'Three_RF',
       'Housing_Units', 'Hispanic_Pop', 'White_Pop', 'Black_Pop', 'Native_Pop',
       'Asian_Pop', 'Pacific_Islander_Pop', 'Other_race_Pop',
       'Bi_Tri_Racial_Pop', 'Male_Pop', 'Female_Pop', 'Veteran',
       'Gini_Ind_Income', 'Rural_Pop', 'Median_Age_Pop', 'Age_65+_Pop',
       'Disability_Pop', 'Below_Poverty_Level', 'Single_Mothers_Pop',
       'Single_Fathers_Pop', '1.51_plus_family_homes', 'HighSchool_Grad',
       'Multilingual_5yrs_Plus', 'Full_time_workers', 'No_Health_Insur',
       'Internet_homes', 'No_Vehicle', 'Homeowner_Vacancy', 'Rental_Vacancy'],
      dtype='object')

In [21]:
# Renaming columns
ndf = JHU_CRE.rename(columns={'FIPS': 'fips', 'Lat': 'lat', 'Long': 'long', 'State': 'state', 'County': 'county', 'Cases': 'cases', 'Deaths': 'deaths', 'POPUNI': 'popuni', 'Total_Population': 'total_population',
                              'Zero_RF': 'zero_rf', 'One-Two_RF': 'one_two_rf', 'Three_RF': 'three_rf', 'Housing_Units': 'housing_units', 'Hispanic_Pop': 'hispanic_pop', 'White_Pop': 'white_pop', 'Black_Pop': 'black_pop', 'Native_Pop': 'native_pop',
                              'Asian_Pop': 'asian_pop', 'Pacific_Islander_Pop': 'pacific_islander_pop', 'Other_race_Pop': 'other_race_pop', 'Bi_Tri_Racial_Pop': 'bi_tri_racial_pop', 'Male_Pop': 'male_pop', 'Female_Pop': 'female_pop', 'Veteran': 'veteran',
                              'Gini_Ind_Income': 'gini_ind_income', 'Rural_Pop': 'rural_pop', 'Median_Age_Pop': 'median_age_pop', 'Age_65+_Pop': 'elder_pop', 'Disability_Pop': 'disability_pop', 'Below_Poverty_Level': 'below_poverty_level', 'Single_Mothers_Pop': 'single_mothers_pop',
                              'Single_Fathers_Pop': 'single_fathers_pop', '1.51_plus_family_homes': 'plus_family_homes', 'HighSchool_Grad': 'highschool_grad', 'Multilingual_5yrs_Plus': 'multilingual_5yrs_plus', 'Full_time_workers': 'full_time_workers', 'No_Health_Insur': 'no_health_insur',
                              'Internet_homes': 'internet_homes', 'No_Vehicle': 'no_vehicle', 'Homeowner_Vacancy': 'homeowner_vacancy', 'Rental_Vacancy': 'rental_vacancy'})

In [22]:
ndf.head()

,fips,cases,deaths,lat,long,state,county,popuni,total_population,zero_rf,...,single_fathers_pop,plus_family_homes,highschool_grad,multilingual_5yrs_plus,full_time_workers,no_health_insur,internet_homes,no_vehicle,homeowner_vacancy,rental_vacancy
0,1001.0,19732,230,32.539527,-86.644082,Alabama,Autauga County,55688,55380,20573,...,723.0,111.0,49283.0,779.0,29291.0,3953.0,44884.0,2450.0,779.0,1726.0
1,1003.0,69641,724,30.727750,-87.722071,Alabama,Baldwin County,221898,212830,78622,...,2218.0,887.0,201483.0,3994.0,112058.0,19748.0,181512.0,7322.0,7322.0,96747.0
2,1005.0,7451,103,31.868263,-85.387129,Alabama,Barbour County,22023,25361,5024,...,220.0,132.0,16120.0,572.0,8170.0,2488.0,13323.0,2246.0,836.0,1629.0
3,1007.0,8067,109,32.996421,-87.125115,Alabama,Bibb County,20393,22493,6280,...,346.0,163.0,16130.0,265.0,8096.0,2182.0,14111.0,1264.0,305.0,1142.0
4,1009.0,18616,261,33.982109,-86.567906,Alabama,Blount County,57697,57681,18189,...,1038.0,115.0,46446.0,1961.0,27463.0,6231.0,42118.0,2365.0,403.0,4846.0


In [23]:
# checking if rename worked
ndf.columns

Index(['fips', 'cases', 'deaths', 'lat', 'long', 'state', 'county', 'popuni',
       'total_population', 'zero_rf', 'one_two_rf', 'three_rf',
       'housing_units', 'hispanic_pop', 'white_pop', 'black_pop', 'native_pop',
       'asian_pop', 'pacific_islander_pop', 'other_race_pop',
       'bi_tri_racial_pop', 'male_pop', 'female_pop', 'veteran',
       'gini_ind_income', 'rural_pop', 'median_age_pop', 'elder_pop',
       'disability_pop', 'below_poverty_level', 'single_mothers_pop',
       'single_fathers_pop', 'plus_family_homes', 'highschool_grad',
       'multilingual_5yrs_plus', 'full_time_workers', 'no_health_insur',
       'internet_homes', 'no_vehicle', 'homeowner_vacancy', 'rental_vacancy'],
      dtype='object')

In [24]:
# checking datatypes
ndf.dtypes

fips                      float64
cases                       int64
deaths                      int64
lat                       float64
long                      float64
state                      object
county                     object
popuni                      int64
total_population            int64
zero_rf                     int64
one_two_rf                  int64
three_rf                    int64
housing_units               int64
hispanic_pop              float64
white_pop                 float64
black_pop                 float64
native_pop                float64
asian_pop                 float64
pacific_islander_pop      float64
other_race_pop            float64
bi_tri_racial_pop         float64
male_pop                  float64
female_pop                float64
veteran                   float64
gini_ind_income           float64
rural_pop                 float64
median_age_pop            float64
elder_pop                 float64
disability_pop            float64
below_poverty_

In [25]:
# casting values as integers
JHU_CRE2 = ndf.astype({"fips": int, "zero_rf": int, "one_two_rf": int, "three_rf": int,
                       "hispanic_pop": int, "white_pop": int, "black_pop": int, "native_pop": int, "asian_pop": int,
                       "pacific_islander_pop": int, "other_race_pop": int, "bi_tri_racial_pop": int, "male_pop": int,
                       "female_pop": int, "veteran": int, "gini_ind_income": int, "rural_pop": int, "median_age_pop": int,
                       "elder_pop": int, "disability_pop": int, "below_poverty_level": int, "single_mothers_pop": int,
                       "single_fathers_pop": int, "plus_family_homes": int, "highschool_grad": int, "multilingual_5yrs_plus": int,
                       "full_time_workers": int, "no_health_insur": int, "internet_homes": int, "no_vehicle": int, "homeowner_vacancy": int,
                       "rental_vacancy": int, "housing_units": int})

In [26]:
JHU_CRE2.dtypes

fips                        int32
cases                       int64
deaths                      int64
lat                       float64
long                      float64
state                      object
county                     object
popuni                      int64
total_population            int64
zero_rf                     int32
one_two_rf                  int32
three_rf                    int32
housing_units               int32
hispanic_pop                int32
white_pop                   int32
black_pop                   int32
native_pop                  int32
asian_pop                   int32
pacific_islander_pop        int32
other_race_pop              int32
bi_tri_racial_pop           int32
male_pop                    int32
female_pop                  int32
veteran                     int32
gini_ind_income             int32
rural_pop                   int32
median_age_pop              int32
elder_pop                   int32
disability_pop              int32
below_poverty_

In [27]:
JHU_CRE2.to_csv('JHU_Cases_Deaths_with_CRE_Data_2_0.csv', index = False)